In [1]:
import json
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.utils import pad_sequences
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from keras.layers import Masking
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, LSTM, Dense, Dropout
features_d1 = []
labels_d1 = []
# Open file for reading
with open('domain1_train.json', 'r') as f:
    for line in f:
        # Parse the JSON line into a Python dictionary
        obj = json.loads(line)
        features_d1.append(obj['text'])
        labels_d1.append(obj['label'])
features_d2 = []
labels_d2 = []
# Open file for reading
with open('domain2_train.json', 'r') as f:
    for line in f:
        # Parse the JSON line into a Python dictionary
        obj = json.loads(line)
        features_d2.append(obj['text'])
        labels_d2.append(obj['label'])
features = features_d1 + features_d2
labels = labels_d1 + labels_d2
new_features = []
for text in features:
    new_text = [x + 1 for x in text]
    new_features.append(new_text)

In [2]:

padded_features = pad_sequences(new_features, padding='pre', value=0)
labels = to_categorical(labels, num_classes=2)


In [3]:
X_train, X_test, y_train, y_test = train_test_split(padded_features, labels, test_size=0.2, random_state=42)

In [70]:
len(X_train[1])

1075

In [74]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Embedding, Masking, Conv1D, MaxPooling1D, LSTM, Dense, Dropout, BatchNormalization, Bidirectional, concatenate, GlobalAveragePooling1D
from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras.callbacks import EarlyStopping
vocab_size = 5000  # Adjust based on your actual vocabulary size
embedding_dim = 128


n_classes = 2  # Binary classification



vocab_size = 5000  # Adjust based on your actual vocabulary size
embedding_dim = 128
sequence_length = 250

n_classes = 2  # Binary classification

# Include early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

model = Sequential([
    Embedding(vocab_size, embedding_dim, mask_zero=True),
    Masking(mask_value=0),
    Conv1D(16, 3, activation='relu', kernel_regularizer=l2(0.005)),
    BatchNormalization(),
    MaxPooling1D(4),
    Conv1D(16, 9, activation='relu', kernel_regularizer=l2(0.005)),  # Additional Conv1D layer
    BatchNormalization(),
    MaxPooling1D(4),
    Dropout(0.6),
    Bidirectional(LSTM(8, recurrent_regularizer=l2(0.005), kernel_regularizer=l2(0.005))),
    Dropout(0.6),
    Dense(n_classes, activation='sigmoid', kernel_regularizer=l1(0.005))
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Summary of the model to view the architecture
model.summary()


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, None, 128)         640000    
                                                                 
 masking_4 (Masking)         (None, None, 128)         0         
                                                                 
 conv1d_15 (Conv1D)          (None, None, 16)          6160      
                                                                 
 batch_normalization_15 (Bat  (None, None, 16)         64        
 chNormalization)                                                
                                                                 
 max_pooling1d_15 (MaxPoolin  (None, None, 16)         0         
 g1D)                                                            
                                                                 
 conv1d_16 (Conv1D)          (None, None, 16)         

In [5]:
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dense
from keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Embedding, Masking, Conv1D, MaxPooling1D, LSTM, Dense, Dropout, BatchNormalization, Bidirectional, concatenate, GlobalAveragePooling1D
from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras.callbacks import EarlyStopping
# Constants
vocab_size = 5000  # Adjust based on your actual vocabulary size
embedding_dim = 128
sequence_length = 250
n_classes = 2  # Assuming binary classification

early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

# Model architecture
model = Sequential([
    Embedding(vocab_size, embedding_dim, mask_zero=True),
    Masking(mask_value=0.0),
    Bidirectional(LSTM(16, return_sequences=True, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01))),
    Dropout(0.5),  # 50% dropout
    Bidirectional(LSTM(8, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01))),
    Dropout(0.5),  # 50% dropout
    Dense(n_classes, activation='sigmoid')
])

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 128)         640000    
                                                                 
 masking_1 (Masking)         (None, None, 128)         0         
                                                                 
 bidirectional (Bidirectiona  (None, None, 32)         18560     
 l)                                                              
                                                                 
 dropout (Dropout)           (None, None, 32)          0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 16)               2624      
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 16)                0

In [7]:
model.fit(X_train, y_train, validation_split=0.2, epochs=50, callbacks=[early_stopping], batch_size = 64)

Epoch 1/50
344/344 [==============================] - 271s 755ms/step - loss: 1.0655 - accuracy: 0.8000 - val_loss: 0.3947 - val_accuracy: 0.8897
Epoch 2/50
344/344 [==============================] - 246s 717ms/step - loss: 0.3850 - accuracy: 0.8952 - val_loss: 0.3482 - val_accuracy: 0.8912
Epoch 3/50
344/344 [==============================] - 245s 714ms/step - loss: 0.3392 - accuracy: 0.9065 - val_loss: 0.3494 - val_accuracy: 0.8841
Epoch 4/50
344/344 [==============================] - 246s 716ms/step - loss: 0.3111 - accuracy: 0.9137 - val_loss: 0.3476 - val_accuracy: 0.8843
Epoch 5/50
344/344 [==============================] - 483s 1s/step - loss: 0.2926 - accuracy: 0.9196 - val_loss: 0.3709 - val_accuracy: 0.8741
Epoch 6/50
344/344 [==============================] - 249s 726ms/step - loss: 0.2730 - accuracy: 0.9259 - val_loss: 0.3539 - val_accuracy: 0.8834


In [14]:
length = [len(text) for text in test_texts]

In [8]:
test_ids = []
test_texts = []
# Open file for reading
with open('test_set.json', 'r') as f:
    for line in f:
        # Parse the JSON line into a Python dictionary
        obj = json.loads(line)
        test_ids.append(obj['id'])
        test_texts.append(obj['text'])
test_features = pad_sequences(test_texts, padding='pre', value=0)
import numpy as np
# Make predictions
predictions = model.predict(test_features)

# Interpret predictions
final_predictions = np.argmax(predictions, axis=1)

print(final_predictions)

32/32 [==============================] - 7s 99ms/step
[0 0 1 1 0 1 0 0 0 0 0 1 0 0 0 1 1 0 1 1 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 1
 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 1 1
 0 0 1 0 1 1 0 0 0 1 1 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0
 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 1 0 1 1 0 1 1 0 1 1 1 0 0 0 0
 0 1 0 0 0 0 1 0 0 0 0 1 0 0 1 0 1 0 1 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0
 0 1 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1
 1 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 1 0 1 0 0 1 0 1 0 0 1 0 0 1 0 1 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 1 0 0 0 0 1
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 1
 1 1 0 1 0 0 0 1 1 1 0 1 1 1 1 0 0 1 0 0 1 1 1 0 0 0 0 1 0 0 1 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 0 1 1 0 1 1 0 0 1 0 0 0 0 0 1 0 0 1
 0 0 0 0 0 1 1 0 0 1 0 0 0 1 1 1 0 0 0 0 0 1 1

In [9]:
import pandas as pd
output_df = pd.DataFrame({"id":test_ids, "class": final_predictions})

In [10]:
output_df.to_csv("output.csv", index=False)